<h1>Lab 1: Basic Digital Signal Processing</h1>

# Initialization

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog

# Exercise 1. *Make a program that solves problem 1 in the following sense: you hard code a sparse trigonometric polynomial (say $p(x) = 2 sin(x) + sin(2x) + 21 sin(300x))$ and then try to guess it by evaluating it on real points only $30$ or $40$ times taken at random in $[0, 400]$. You only know that it is a trigonometric polynomial of degree smaller than 400 and with few components.*

In this case we are considering a sparse polynomial of maximum degree $N=400$, and we know that the number of non-zero coefficients are $K=3$. We thus consider a system 

$$Ax=b$$

with $A$ being an $N\times M$ matrix. This system is undetermined, with $M<N$, and so we consider the solution $x$ with minimal $\mathcal{l}^1$ norm. As it has been shown in the Lab, doing so allows, in general, to compute a solution with a good number of vanishing coefficients.
More precisely, to solve the problem we can do the following: </br>

* Consider the system $A x=y$ where $A$ is an $N \times M$ matrix. This system represents the problem we want to solve, which encodes the values of the polynomial at the given points. It is highly underdetermined because $M<N$, meaning there are multiple possible solutions.

* Our goal is to choose the solution $x$ that has the minimal $\ell^1$ norm.
Hence, let's assume the presence of a trigonometric polynomial with a degree less than 400, denoted as $p(x)=a_0+\sum_{k=1}^{400} a_k \sin (k x)$. To construct the matrix $A$ and the vector $y$, we start by generating 35 random points within the interval $[0,400]$. The resulting form of the matrix $A$ and the vector $y$ are 

$$
A=\left[\begin{array}{cccc}
1 & \sin \left(t_1 x\right) & \cdots & \sin \left(t_1 x\right) \\
\vdots & \cdots & \cdots & \vdots \\
1 & \sin \left(t_{35} x\right) & \cdots & \sin \left(t_{35} x\right)
\end{array}\right], \quad y=\left[\begin{array}{c}
p\left(t_1\right) \\
\vdots \\
p\left(t_{35}\right)
\end{array}\right]
$$
 
By solving the system $A x=y$ and selecting the solution with the minimal $\ell^1$ norm, we can find the coefficients of our polynomial. As mentioned in the lab, the problem of minimizing $|x|_1$ subject to the constraint $Ax=y$ can be transformed into a linear programming problem. Specifically, we define $x=u-v$ where $u_j=x_j^{+}$ and $v_j=x_j^{-}$. Thus, our objective is to minimize

$$
\sum_{i=1}^N u_i+v_i
$$
 
subject to $A(u-v)=y$ and $u_j \geq 0, v_j \geq 0$. In Python, we can use the standard linear programming solver linprog to solve this problem. First, we define a vector called $c$, which in this case consists of ones since we aim to minimize the given objective. Additionally, we include the matrix $A$ and the vector $y$. Running the solver provides us with the vector $x$, which represents the polynomial coefficients with the smallest $\ell^1$ norm. After the optimization problem finishes, the script displays the non-zero values (coefficients) of $x$ on the screen, which correspond to the expected coefficients.

In [6]:
N = 400
M = 35

def p(x):
  return 2*np.sin(x)+np.sin(2*x)+21*np.sin(300*x)

# Generate M random points between 0 and N
points = np.random.rand(M) * N

In [7]:
# Create the matrix A to solve Ax = y
A = np.zeros((M, N))
for i in range(M):
    for j in range(1, N):
        A[i, 0] = 1
        A[i, j] = np.sin((j-1)*points[i])

# Create the vector y to solve Ax = y
y = np.zeros(M)
for i in range(M):
    y[i] = p(points[i])

# Solve the minimization problem
c = np.ones(N)
result = linprog(c, A_eq=A, b_eq=y)

# Print the values different from 0
print("The coefficients different from 0 are:")
for i, value in enumerate(result.x):
    if abs(value) > 1e-10:
        print("x({}) = {}".format(i-1, value))

The coefficients different from 0 are:
x(1) = 1.9999999999999574
x(2) = 1.0000000000000053
x(300) = 21.000000000000014


As we can see, the obtained values of the coefficients correspond to the one present in the polynomial. (within a small computational error)

# Exercise 2. *For the very brave: Try to reproduce the numerical experiment suggested. There is a program to find the minimum the total variation subject to some linear constrains in the web page of E. Cand`es*